In [1]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw
import re
import os
import pytesseract as pt
from pytesseract import Output
from skimage.segmentation import clear_border
import numpy as np
from PIL import Image, ImageFont, ImageDraw
import json

In [2]:
# Show image using matplotlib. If save is true, it will save and file name will be the var title
def show_images(img, title, save = False):
    fig = plt.gcf() # busca a figura atual
    fig.set_size_inches(10,5)
    plt.axis('off') # remove a visualizacao de eixos
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) # conversao de cores com OpenCV
    if save is True:
        new_image = 'resultados/' + title + '.jpeg'
        cv2.imwrite(new_image, img)
    plt.title(title)
    plt.show() # mostra a imagem

In [3]:
# Import, grayscale
image = cv2.imread("imagens/placa_carro3.jpg")
grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
show_images(grayscale, '00_Grayscale')

In [19]:

# Black Hat
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 13))
black_hat = cv2.morphologyEx(grayscale, cv2.MORPH_BLACKHAT, rect_kernel)
show_images(black_hat, '01_BlackHat')


In [20]:
# Sobel
# 32 bits do tipo float
sobel_x = cv2.Sobel(
    black_hat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=1
)  # Usamos sobel_x por conta da placa ter uma largura maior que a altura
sobel_x = np.absolute(sobel_x)
sobel_x = sobel_x.astype("uint8")
show_images(sobel_x, '02_SobelX')


In [6]:
# Aula 14 - Mascara
# Gaussian Blur + Closing
sobel_x = cv2.GaussianBlur(sobel_x, (5, 5), 0)
sobel_x = cv2.morphologyEx(sobel_x, cv2.MORPH_CLOSE, rect_kernel)
show_images(sobel_x, '03_GaussianBlur_and_Closing')


invalid command name "140440535978432process_stream_events"
    while executing
"140440535978432process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [7]:
# Otsu
value, thresh_otsu = cv2.threshold(sobel_x, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
show_images(thresh_otsu, '04_Otsu')

invalid command name "140440924130752process_stream_events"
    while executing
"140440924130752process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [8]:
# Erosion
square_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
erosion = cv2.erode(thresh_otsu, square_kernel, iterations=2)  # double erosion
show_images(erosion, '06_Double_Erosion')

invalid command name "140443170638720process_stream_events"
    while executing
"140443170638720process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [9]:
# Dilatation
dilatation = cv2.dilate(erosion, square_kernel, iterations=2) # double dilatation
show_images(dilatation, '07_Double_Dilatation')

invalid command name "140441623836416process_stream_events"
    while executing
"140441623836416process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [10]:
# Closing (grayscale image)
closing = cv2.morphologyEx(grayscale, cv2.MORPH_CLOSE, square_kernel)
show_images(closing, '08_Closing')

invalid command name "140441635760512process_stream_events"
    while executing
"140441635760512process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [11]:
value, mask = cv2.threshold(closing, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
show_images(mask, '09_Mask')

can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [12]:
# Threshold
bitwise_and = cv2.bitwise_and(dilatation, dilatation, mask = mask)
show_images(bitwise_and, '10_BitwiseAnd')


can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [13]:
dilatation2 = cv2.dilate(bitwise_and, square_kernel, iterations= 2)
show_images(dilatation2, '11_Dilatation')

can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [14]:
erosion2 = cv2.erode(dilatation2, square_kernel)
show_images(erosion2, '12_Erosion')

invalid command name "140439960716096process_stream_events"
    while executing
"140439960716096process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [15]:
# Aula 15 - Deteccao de caracteres
clear = clear_border(erosion2)
show_images(clear, '13_ClearBorder')

invalid command name "140440534143552process_stream_events"
    while executing
"140440534143552process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [16]:
contours, hierarchy = cv2.findContours(clear, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key = cv2.contourArea, reverse=True)[:10]
print(contours)

[array([[[397, 195]],

       [[397, 197]],

       [[396, 198]],

       [[396, 202]],

       [[397, 203]],

       [[397, 255]],

       [[403, 255]],

       [[405, 253]],

       [[405, 252]],

       [[406, 251]],

       [[446, 251]],

       [[448, 253]],

       [[450, 253]],

       [[450, 249]],

       [[451, 248]],

       [[453, 248]],

       [[454, 249]],

       [[454, 253]],

       [[458, 253]],

       [[460, 251]],

       [[463, 251]],

       [[463, 248]],

       [[464, 247]],

       [[534, 247]],

       [[535, 248]],

       [[535, 252]],

       [[537, 252]],

       [[538, 253]],

       [[540, 253]],

       [[540, 242]],

       [[541, 241]],

       [[543, 241]],

       [[544, 242]],

       [[544, 247]],

       [[545, 248]],

       [[545, 253]],

       [[548, 253]],

       [[549, 252]],

       [[549, 251]],

       [[550, 250]],

       [[550, 248]],

       [[551, 247]],

       [[590, 247]],

       [[591, 248]],

       [[591, 252]],

       [[

In [17]:
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    proportion = float(w)/h
    if proportion >= 3 and proportion <= 3.5:
        license_plate = grayscale[y:y+h, x:x+w]
        value, interest_region = cv2.threshold(license_plate, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        interest_region = clear_border(interest_region)
        show_images(license_plate, '14_Plate')
        show_images(interest_region, '14_Interest_Region')

can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [18]:
# Se atentar ao nome de usuario em --tessdata-dir
text = pt.image_to_string(interest_region, lang='por',  config='--tessdata-dir /home/lanzo/tessdata --psm 6')
print(text)

extracted_text = re.search('\w{3}\d{1}\w{1}\d{2}', text)
print(extracted_text)
print(extracted_text.group(0))
print('Texto Detectado:', extracted_text.group(0))
cv2.imwrite('imagem3_apos_pre_processamento.png', interest_region)

.BDM3D69

<re.Match object; span=(1, 8), match='BDM3D69'>
BDM3D69
Texto Detectado: BDM3D69


True